# 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
master_df = pd.read_csv('./result/wholesale_2022-06-23_preprocessed.csv')

In [3]:
agricproduct_wholesale_df = pd.read_csv('./result/agriproduct_wholesale_df.csv')

In [4]:
master_df

,category_code,p_code,p_group,kind_name_new,rank,unit,price,date
0,AW_A00115,AW_A00115,쌀,쌀(일반계)(20kg),상품,20kg,"44,500",2022-06-23
1,AW_A00116,AW_A00116,쌀,쌀(일반계)(20kg),중품,20kg,"41,000",2022-06-23
2,AW_A00153,AW_A00153,찹쌀,찹쌀(일반계)(40kg),중품,40kg,"97,800",2022-06-23
3,AW_A00198,AW_A00198,콩,흰 콩(국산)(40kg),상품,40kg,"240,000",2022-06-23
4,AW_A00200,AW_A00200,콩,흰 콩(수입)(35kg),중품,35kg,"122,000",2022-06-23
...,...,...,...,...,...,...,...,...
44564,SW_A00020,SW_A00020,건오징어,건오징어(20마리),중품,20마리,"77,000",2021-06-24
44565,SW_A00053,SW_A00053,김,마른김(1속),중품,1속,"6,000",2021-06-24
44566,SW_A00019,SW_A00019,건미역,건미역(1kg),상품,1kg,"10,000",2021-06-24
44567,SW_A00140,SW_A00140,전복,전복(1kg),중품,1kg,"33,000",2021-06-24


In [5]:
product_wholesale_df = pd.DataFrame()
product_wholesale_df['p_code'] = master_df['p_code']
product_wholesale_df['description'] = master_df['kind_name_new']
product_wholesale_df['p_name'] = master_df['p_group']
product_wholesale_df['c_code_agrr'] = None
product_wholesale_df['c_code_agrw'] = master_df['category_code']
product_wholesale_df['m_coder'] = None
product_wholesale_df['m_codew'] = 'W1'

In [6]:
product_wholesale_df

,p_code,description,p_name,c_code_agrr,c_code_agrw,m_coder,m_codew
0,AW_A00115,쌀(일반계)(20kg),쌀,None,AW_A00115,None,W1
1,AW_A00116,쌀(일반계)(20kg),쌀,None,AW_A00116,None,W1
2,AW_A00153,찹쌀(일반계)(40kg),찹쌀,None,AW_A00153,None,W1
3,AW_A00198,흰 콩(국산)(40kg),콩,None,AW_A00198,None,W1
4,AW_A00200,흰 콩(수입)(35kg),콩,None,AW_A00200,None,W1
...,...,...,...,...,...,...,...
44564,SW_A00020,건오징어(20마리),건오징어,None,SW_A00020,None,W1
44565,SW_A00053,마른김(1속),김,None,SW_A00053,None,W1
44566,SW_A00019,건미역(1kg),건미역,None,SW_A00019,None,W1
44567,SW_A00140,전복(1kg),전복,None,SW_A00140,None,W1


In [7]:
product_wholesale_df.isnull().sum()

p_code             0
description        0
p_name             0
c_code_agrr    44569
c_code_agrw        0
m_coder        44569
m_codew            0
dtype: int64

In [8]:
product_wholesale_df.to_csv('./result/product_wholesale.csv', encoding='utf8', index=False)

In [9]:
product_wholesale_df = product_wholesale_df.drop_duplicates().reset_index(drop=True)

In [10]:
product_wholesale_df

,p_code,description,p_name,c_code_agrr,c_code_agrw,m_coder,m_codew
0,AW_A00115,쌀(일반계)(20kg),쌀,None,AW_A00115,None,W1
1,AW_A00116,쌀(일반계)(20kg),쌀,None,AW_A00116,None,W1
2,AW_A00153,찹쌀(일반계)(40kg),찹쌀,None,AW_A00153,None,W1
3,AW_A00198,흰 콩(국산)(40kg),콩,None,AW_A00198,None,W1
4,AW_A00200,흰 콩(수입)(35kg),콩,None,AW_A00200,None,W1
...,...,...,...,...,...,...,...
196,SW_A00053,마른김(1속),김,None,SW_A00053,None,W1
197,SW_A00019,건미역(1kg),건미역,None,SW_A00019,None,W1
198,SW_A00140,전복(1kg),전복,None,SW_A00140,None,W1
199,SW_A00107,새우(흰다리)(수입),새우,None,SW_A00107,None,W1


In [11]:
import pymysql

db = pymysql.connect(host="db-1.cnt4cbkbsgdt.us-east-1.rds.amazonaws.com",
                     port=3306,
                     user='admin',
                     passwd='12345678',
                     db='public_api',
                     charset='utf8')

cursor = db.cursor()

# 상품 데이터 삽입
for idx, row in product_wholesale_df.iterrows():

    sql = f"INSERT INTO product VALUES('{row['p_code']}','{row['description']}','{row['p_name']}', NULL, '{row['c_code_agrw']}', NULL, '{row['m_codew']}')"
    cursor.execute(sql)
    db.commit()

    if idx % 100 == 0:
        print(f"{idx} 번째 row insert 완료")

0 번째 row insert 완료
100 번째 row insert 완료
200 번째 row insert 완료
